<a href="https://colab.research.google.com/github/kim10367/Sunny-Projects/blob/main/Supply_Chain_Analytics_Individual_Assignment_(SUNNY_KIM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tabulate

In [3]:
import numpy as np
from tabulate import tabulate
import math

1. individual sourcing

In [4]:
# Given data for all products
products = {
    1: {'demand': 1000, 'common_cost': 100, 'specific_cost': 10, 'unit_cost': 50, 'holding_rate': 0.2},
    2: {'demand': 300, 'common_cost': 100, 'specific_cost': 20, 'unit_cost': 60, 'holding_rate': 0.2},
    3: {'demand': 100, 'common_cost': 100, 'specific_cost': 25, 'unit_cost': 30, 'holding_rate': 0.2},
    4: {'demand': 50, 'common_cost': 100, 'specific_cost': 25, 'unit_cost': 30, 'holding_rate': 0.2},
}

# Function to calculate optimal Q
def calculate_eoq(demand, ordering_cost, holding_cost):
    return math.sqrt((2 * demand * ordering_cost) / holding_cost)

# Function to calculate the total cost for a product
def calculate_total_cost(demand, eoq, ordering_cost, holding_cost):
    return (demand / eoq) * ordering_cost + (eoq / 2) * holding_cost

# Calculate and sum individual sourcing costs
total_cost = 0
for product_id, product_info in products.items():
    ordering_cost = product_info['common_cost'] + product_info['specific_cost']
    holding_cost = product_info['holding_rate'] * product_info['unit_cost']
    eoq = calculate_eoq(product_info['demand'], ordering_cost, holding_cost)
    total_cost += calculate_total_cost(product_info['demand'], eoq, ordering_cost, holding_cost)

print(f"Total annual operational cost for individual sourcing: {total_cost}")


Total annual operational cost for individual sourcing: 3073.9153138822376


2. complete aggregation

In [7]:
# Initializing data as numpy arrays
demands = np.array([1000, 300, 100, 50])
common_costs = np.array([100, 100, 100, 100])
specific_costs = np.array([10, 20, 25, 25])
unit_costs = np.array([50, 60, 30, 30])
holding_cost_rates = np.array([0.2, 0.2, 0.2, 0.2])

# Average of ordering costs for complete aggregation
S = np.mean(common_costs + specific_costs)

# Calculate the numerator of the formula for n
numerator = np.sum(demands * holding_cost_rates * unit_costs)

# Calculate n for complete aggregation
n = np.sqrt(numerator / (2 * S))

# Calculate total holding cost and ordering cost under complete aggregation
Optimal_Q = demands / n
Total_Holding_Cost = np.sum((Optimal_Q / 2) * holding_cost_rates * unit_costs)
Total_Ordering_Cost = n * S

# Total Annual Cost
Total_Annual_Cost_2 = Total_Ordering_Cost + Total_Holding_Cost

# Display results
print(f"Order Frequency (n) under Complete Aggregation: {n:.2f}")
print(f"Total Annual Cost under Complete Aggregation: ${Total_Annual_Cost_2:.2f}")

# For displaying each product's details in a table
opt_results_aggregated = [["Product {}".format(i+1), Optimal_Q[i], demands[i] / Optimal_Q[i], n, (Optimal_Q[i] / 2) * holding_cost_rates[i] * unit_costs[i]] for i in range(len(demands))]
headers_aggregated = ["Product", "Optimal Q", "Time Between Orders (T)", "Order Frequency (N)", "Holding Cost"]

# Print the table for Complete Aggregation
print(tabulate(opt_results_aggregated, headers=headers_aggregated, tablefmt="pretty", numalign="right"))


Order Frequency (n) under Complete Aggregation: 7.77
Total Annual Cost under Complete Aggregation: $1865.48
+-----------+--------------------+-------------------------+---------------------+--------------------+
|  Product  |     Optimal Q      | Time Between Orders (T) | Order Frequency (N) |    Holding Cost    |
+-----------+--------------------+-------------------------+---------------------+--------------------+
| Product 1 | 128.65350418053538 |    7.772815877574013    |  7.772815877574012  | 643.2675209026769  |
| Product 2 | 38.596051254160614 |    7.772815877574013    |  7.772815877574012  | 231.5763075249637  |
| Product 3 | 12.865350418053538 |    7.772815877574013    |  7.772815877574012  | 38.596051254160614 |
| Product 4 | 6.432675209026769  |    7.772815877574013    |  7.772815877574012  | 19.298025627080307 |
+-----------+--------------------+-------------------------+---------------------+--------------------+


3. tailored aggregation

In [23]:
# Given product data (assuming common_cost is defined outside the product dictionaries)
common_cost = 100

# Step 1: Identify the most frequently ordered product
eoqs = {pid: calculate_eoq(product['demand'], product['specific_cost'], product['holding_rate'] * product['unit_cost']) for pid, product in products.items()}
most_frequent_product = min(eoqs, key=eoqs.get)  # Product with the smallest EOQ (most frequently ordered)

# Step 2 & 3: Recompute frequencies for other products and identify mi
m_values = {pid: math.ceil(max(eoqs.values()) / eoq) for pid, eoq in eoqs.items()}  # mi values, rounded up

# Step 4: Recalculate ordering frequency n for the most frequently ordered product
sum_hCFm = sum(products[pid]['holding_rate'] * products[pid]['unit_cost'] * products[pid]['demand'] * m for pid, m in m_values.items())
total_specific_costs = sum(products[pid]['specific_cost'] / m for pid, m in m_values.items())
n = math.sqrt(sum_hCFm / (2 * (common_cost + total_specific_costs)))

# Step 5: Identify ordering frequency for all products using n and mi
frequencies = {pid: n / m for pid, m in m_values.items()}

# Calculate the total annual operational cost for the tailored aggregation strategy
total_annual_cost_tailored = 0

for pid, product in products.items():
    # Calculate the order quantity for each product
    Q = product['demand'] / frequencies[pid]

    # Calculate the ordering cost for each product
    # For the most frequently ordered product, use common_cost + specific_cost
    # For others, use specific_cost as ordering cost
    if pid == most_frequent_product:
        ordering_cost = common_cost + product['specific_cost']
    else:
        ordering_cost = product['specific_cost']

    ordering_cost_annual = frequencies[pid] * ordering_cost

    # Calculate the holding cost for each product
    holding_cost = (Q / 2) * product['holding_rate'] * product['unit_cost']

    # Sum the ordering and holding costs for the total annual cost
    total_annual_cost_tailored += ordering_cost_annual + holding_cost

print(f"Total annual operational cost under the tailored aggregation strategy: {total_annual_cost_tailored}")


Total annual operational cost under the tailored aggregation strategy: 1779.7106007677785
